In [1]:
from pathlib import Path
import pandas as pd
pd.options.mode.copy_on_write = True
pd.set_option('display.max_colwidth', 20)   # JSON columns make things look weird in notebook without this

from datatype_recovery.models.dataset import load_dataset_from_path
from datatype_recovery.models.dataset.encoding import *

In [2]:
dataset_path = Path.home()/'datasets/coreutils_full_5hops'
dataset = load_dataset_from_path(dataset_path)

In [3]:
df = dataset.read_vars_csv()
df

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,LocRegName_Debug,LocOffset_Debug,TypeCategory_Debug,TypeSeq_Debug,HasDWARF,TypeJson_Debug,BinaryId,Vartype,IsReturnType_Debug,IsReturnType_Strip
0,1056281,"37,78,82,107,111",local_a8,rusage,stack,NaN,-168.0,STRUCT,STRUCT,usage,...,NaN,-168.0,STRUCT,STRUCT,True,"{""kind"": ""Struct...",1000,l,False,False
1,1056439,"37,94,114",local_b8,timeval,stack,NaN,-184.0,STRUCT,STRUCT,real_end,...,NaN,-184.0,STRUCT,STRUCT,True,"{""kind"": ""Struct...",1000,l,False,False
2,1056439,"57,160,180,226,246",local_a8,rusage,stack,NaN,-168.0,STRUCT,STRUCT,usage,...,NaN,-168.0,STRUCT,STRUCT,True,"{""kind"": ""Struct...",1000,l,False,False
3,1056881,"300,300,300",auVar1,uchar[16],unique,NaN,243584.0,ARR,"ARR,uchar",auVar2,...,NaN,243584.0,ARR,"ARR,uchar",False,"{""kind"": ""ArrayT...",1000,l,False,False
4,1056881,"145,485",iVar2,int32,register,eax,0.0,BUILTIN,int32,iVar3,...,eax,0.0,BUILTIN,int32,False,"{""kind"": ""Builti...",1000,l,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85154,1092028,-1,NaN,uint32,NaN,NaN,NaN,BUILTIN,uint32,NaN,...,NaN,NaN,BUILTIN,char,True,"{""kind"": ""Builti...",1116,p,True,True
85155,1092192,-1,NaN,char*,NaN,NaN,NaN,PTR,"PTR,char",NaN,...,NaN,NaN,PTR,"PTR,char",True,"{""kind"": ""Pointe...",1116,p,True,True
85156,1092232,-1,NaN,uint64,NaN,NaN,NaN,BUILTIN,uint64,NaN,...,NaN,NaN,BUILTIN,int32,True,"{""kind"": ""Builti...",1116,p,True,True
85157,1092416,-1,NaN,void,NaN,NaN,NaN,BUILTIN,void,NaN,...,NaN,NaN,BUILTIN,int32,True,"{""kind"": ""Builti...",1116,p,True,True


In [4]:
df.TypeJson_Debug

dt = datatype_from_json_str(df.TypeJson_Debug.iloc[3])
dt.type_sequence[-1].category

# TODO: category
# - PRIMITIVE, STRUCT, UNION, FUNC[, ENUM?]
# TODO: size
# - 0, 1, 2, 4, 8[, 10?], 16[, >16?]
# --> long double (10) can be represented as 16 since we have no 16B floating point
# --> >16 can be represented as 16
# TODO: floating (0 or 1)
# TODO: signed (0 or 1)


'BUILTIN'

In [5]:
df = df.loc[df.TypeCategory_Debug!='COMP',:]

In [6]:
df['DebugType'] = df[(df.TypeCategory_Debug!='COMP')].TypeJson_Debug.apply(lambda x: datatype_from_json_str(x))

In [7]:
df.DebugType.iloc[0].leaf_type.category
df['LeafCategory'] = df.DebugType.apply(lambda x: x.leaf_type.category)
df['LeafSigned'] = df.DebugType.apply(lambda x: x.leaf_type.is_signed)
df['LeafFloating'] = df.DebugType.apply(lambda x: x.leaf_type.is_floating)
df['LeafSize'] = df.DebugType.apply(lambda x: x.leaf_type.primitive_size)

In [8]:
df.groupby('LeafCategory').count()

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,HasDWARF,TypeJson_Debug,BinaryId,Vartype,IsReturnType_Debug,IsReturnType_Strip,DebugType,LeafSigned,LeafFloating,LeafSize
LeafCategory,,,,,,,,,,,,,,,,,,,,,
BUILTIN,56366,56366,41369,56366,41369,26888,41369,56366,56366,41369,...,56366,56366,56366,56366,56366,56366,56366,56366,56366,56366
ENUM,2329,2329,1640,2329,1640,1129,1640,2329,2329,1640,...,2329,2329,2329,2329,2329,2329,2329,2329,2329,2329
FUNC,256,256,256,256,256,184,256,256,256,256,...,256,256,256,256,256,256,256,256,256,256
STRUCT,10327,10327,9589,10327,9589,6206,9589,10327,10327,9589,...,10327,10327,10327,10327,10327,10327,10327,10327,10327,10327
UNION,30,30,30,30,30,5,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30


In [9]:
df.groupby('LeafSize').count()

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,HasDWARF,TypeJson_Debug,BinaryId,Vartype,IsReturnType_Debug,IsReturnType_Strip,DebugType,LeafCategory,LeafSigned,LeafFloating
LeafSize,,,,,,,,,,,,,,,,,,,,,
0,19548,19548,13511,19548,13511,9259,13511,19548,19548,13511,...,19548,19548,19548,19548,19548,19548,19548,19548,19548,19548
1,20710,20710,14527,20710,14527,8522,14527,20710,20710,14527,...,20710,20710,20710,20710,20710,20710,20710,20710,20710,20710
2,356,356,298,356,298,252,298,356,356,298,...,356,356,356,356,356,356,356,356,356,356
4,15628,15628,12813,15628,12813,9429,12813,15628,15628,12813,...,15628,15628,15628,15628,15628,15628,15628,15628,15628,15628
8,12997,12997,11684,12997,11684,6916,11684,12997,12997,11684,...,12997,12997,12997,12997,12997,12997,12997,12997,12997,12997
16,69,69,51,69,51,34,51,69,69,51,...,69,69,69,69,69,69,69,69,69,69


In [45]:
#df.DebugType.apply(lambda x: x.type_sequence_str.split(',') + ['L']*(3-len(x.type_sequence_str.split(','))))
#df.DebugType.apply(lambda x: x.type_sequence_str.split(',')[-1])


df['PtrHier'] = df.DebugType.apply(lambda x: ''.join(x.ptr_hierarchy(3)))
df['PtrL1'] = df.PtrHier.apply(lambda x: x[0])
df['PtrL2'] = df.PtrHier.apply(lambda x: x[1])
df['PtrL3'] = df.PtrHier.apply(lambda x: x[2])
df

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,IsReturnType_Strip,DebugType,LeafCategory,LeafSigned,LeafFloating,LeafSize,PtrHier,PtrL1,PtrL2,PtrL3
0,1056281,"37,78,82,107,111",local_a8,rusage,stack,NaN,-168.0,STRUCT,STRUCT,usage,...,False,rusage,STRUCT,False,False,0,LLL,L,L,L
1,1056439,"37,94,114",local_b8,timeval,stack,NaN,-184.0,STRUCT,STRUCT,real_end,...,False,timeval,STRUCT,False,False,0,LLL,L,L,L
2,1056439,"57,160,180,226,246",local_a8,rusage,stack,NaN,-168.0,STRUCT,STRUCT,usage,...,False,rusage,STRUCT,False,False,0,LLL,L,L,L
3,1056881,"300,300,300",auVar1,uchar[16],unique,NaN,243584.0,ARR,"ARR,uchar",auVar2,...,False,uchar[16],BUILTIN,False,False,1,ALL,A,L,L
4,1056881,"145,485",iVar2,int32,register,eax,0.0,BUILTIN,int32,iVar3,...,False,int32,BUILTIN,True,False,4,LLL,L,L,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85154,1092028,-1,NaN,uint32,NaN,NaN,NaN,BUILTIN,uint32,NaN,...,True,char,BUILTIN,True,False,1,LLL,L,L,L
85155,1092192,-1,NaN,char*,NaN,NaN,NaN,PTR,"PTR,char",NaN,...,True,char*,BUILTIN,True,False,1,PLL,P,L,L
85156,1092232,-1,NaN,uint64,NaN,NaN,NaN,BUILTIN,uint64,NaN,...,True,int32,BUILTIN,True,False,4,LLL,L,L,L
85157,1092416,-1,NaN,void,NaN,NaN,NaN,BUILTIN,void,NaN,...,True,int32,BUILTIN,True,False,4,LLL,L,L,L


In [60]:
len(df[(df.PtrL3!='L')])/len(df)*100
df[(df.PtrL3!='L')]

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,IsReturnType_Strip,DebugType,LeafCategory,LeafSigned,LeafFloating,LeafSize,PtrHier,PtrL1,PtrL2,PtrL3
4921,1069103,"268,752,1798,193...",alStack_198,int64[8],stack,NaN,-408.0,ARR,"ARR,int64",all_line,...,False,linebuffer*[2][4],STRUCT,False,False,0,AAP,A,A,P
8428,1102723,"8,194,199,199,20...",pppcVar4,char***,register,rsp,32.0,PTR,"PTR,PTR,PTR,char",pppuVar4,...,False,uchar***,BUILTIN,False,False,1,PPP,P,P,P
8429,1102723,"8,282,300,714,739",pppcVar5,char***,register,rsp,32.0,PTR,"PTR,PTR,PTR,char",pppuVar5,...,False,uchar***,BUILTIN,False,False,1,PPP,P,P,P
13068,1101544,"8,194,199,199,20...",pppcVar4,char***,register,rsp,32.0,PTR,"PTR,PTR,PTR,char",pppuVar4,...,False,uchar***,BUILTIN,False,False,1,PPP,P,P,P
13069,1101544,"8,282,300,714,739",pppcVar5,char***,register,rsp,32.0,PTR,"PTR,PTR,PTR,char",pppuVar5,...,False,uchar***,BUILTIN,False,False,1,PPP,P,P,P
27527,1080339,"97,114",puVar1,uint64*,unique,NaN,13312.0,PTR,"PTR,uint64",pppcVar1,...,False,char***,BUILTIN,True,False,1,PPP,P,P,P
58306,1116893,354,param_5,int64*,register,r8,128.0,PTR,"PTR,int64",tokens_out,...,False,char***,BUILTIN,True,False,1,PPP,P,P,P
58769,1081498,354,param_5,int64*,register,r8,128.0,PTR,"PTR,int64",tokens_out,...,False,char***,BUILTIN,True,False,1,PPP,P,P,P
76604,1075020,292,param_3,uint64**,register,rdx,16.0,PTR,"PTR,PTR,uint64",pline,...,False,char***,BUILTIN,True,False,1,PPP,P,P,P
78850,1114560,"48,55",param_3,int64*,register,rdx,16.0,PTR,"PTR,int64",pfps,...,False,FILE***,STRUCT,False,False,0,PPP,P,P,P


In [39]:
print(hex(1069857))
df[df.LeafSize==3]

0x105321


,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,TypeJson_Debug,BinaryId,Vartype,IsReturnType_Debug,IsReturnType_Strip,DebugType,LeafCategory,LeafSigned,LeafFloating,LeafSize


In [14]:
bdf = dataset.read_binaries_csv()
bdf[bdf.BinaryId==1043]

,BinaryId,Name,RunGid,OrigBinaryId
43,1043,who,0,43


In [42]:
df.groupby(['LeafCategory','LeafSigned','LeafFloating']).count()

FunctionStart  Signature  Name_Strip   
LeafCategory LeafSigned LeafFloating                                         
BUILTIN      False      False                 20105      20105       14232  \
             True       False                 35977      35977       26892   
                        True                    284        284         245   
ENUM         False      False                  2329       2329        1640   
FUNC         False      False                   256        256         256   
STRUCT       False      False                 10327      10327        9589   
UNION        False      False                    30         30          30   

                                      Type_Strip  LocType_Strip   
LeafCategory LeafSigned LeafFloating                              
BUILTIN      False      False              20105          14232  \
             True       False              35977          26892   
                        True                 284            245   
ENUM         False      False               2329           1640   
FUNC         False      False                256            256   
STRUCT       False      False              10327           9589   
UNION        False      False                 30             30   

                                      LocRegName_Strip  LocOffset_Strip   
LeafCategory LeafSigned LeafFloating                                      
BUILTIN      False      False                     8444            14232  \
             True       False                    18237            26892   
                        True                       207              245   
ENUM         False      False                     1129             1640   
FUNC         False      False                      184              256   
STRUCT       False      False                     6206             9589   
UNION        False      False                        5               30   

                                      TypeCategory_Strip  TypeSeq_Strip   
LeafCategory LeafSigned LeafFloating                                      
BUILTIN      False      False                      20105          20105  \
             True       False                      35977          35977   
                        True                         284            284   
ENUM         False      False                       2329           2329   
FUNC         False      False                        256            256   
STRUCT       False      False                      10327          10327   
UNION        False      False                         30             30   

                                      Name_Debug  ...  LocOffset_Debug   
LeafCategory LeafSigned LeafFloating              ...                    
BUILTIN      False      False              14232  ...            14232  \
             True       False              26892  ...            26892   
                        True                 245  ...              245   
ENUM         False      False               1640  ...             1640   
FUNC         False      False                256  ...              256   
STRUCT       False      False               9589  ...             9589   
UNION        False      False                 30  ...               30   

                                      TypeCategory_Debug  TypeSeq_Debug   
LeafCategory LeafSigned LeafFloating                                      
BUILTIN      False      False                      20105          20105  \
             True       False                      35977          35977   
                        True                         284            284   
ENUM         False      False                       2329           2329   
FUNC         False      False                        256            256   
STRUCT       False      False                      10327          10327   
UNION        False      False                         30             30   

                                    

In [59]:
df['BuiltinSizes'] = df[(df.TypeCategory_Debug=='BUILTIN')].DebugType.apply(lambda x: x.size)

In [61]:
df.groupby('BuiltinSizes').count()[['FunctionStart']]

,FunctionStart
BuiltinSizes,
0.0,3444
1.0,6569
2.0,125
4.0,13119
6.0,3
8.0,12091
10.0,63
16.0,1
